# Computer Vision (Winter 2018/19)

## Practice Session 04: Color, Camera

November 28th, 2018

Ulf Krumnack

Institute of Cognitive Science
University of Osnabrück

## Today's Session

* discussen exercise sheet03
* new exercise sheet04
* color
* camera

# Image Gradients

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage.filters import sobel,laplace,gaussian_filter
from skimage import data

img = data.camera().astype(np.float32)/256

# cut out a line from the image
row = 100
c = np.arange(img.shape[1])
r = img[row,:].copy()
img[row-1:row+1,:] = 1-r

# plot the image
plt.figure(figsize=(12, 10))
plt.subplot(2,2,1); plt.gray(); plt.imshow(img)

# plot line as function
plt.subplot(2,2,2);
smooth = gaussian_filter(r,3)
plt.plot(c,r,'b', c,smooth,'g')

plt.subplot(2,2,3);
first = sobel(r)
first_smooth = sobel(smooth)
plt.plot(c,r,'y', c,first,'r', c,first_smooth,'b')

plt.subplot(2,2,4);
second = laplace(r)
second_smooth = laplace(smooth)
plt.plot(c,r,'y', c,second,'r', c,second_smooth,'b')

plt.show()

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage.filters import sobel,laplace,gaussian_filter
from skimage import data

img = data.camera().astype(np.float32)/256

img_smooth = gaussian_filter(img,3)

grad_x = sobel(img, axis=1)
grad_y = sobel(img, axis=0)

grad_abs = np.sqrt(grad_x**2 + grad_y**2)

# plot the image
plt.figure(figsize=(12, 36))
plt.subplot(6,2,1); plt.gray(); plt.imshow(img)

plt.subplot(6,2,3); plt.gray(); plt.imshow(img_smooth)
plt.subplot(6,2,5); plt.gray(); plt.imshow(grad_x)
plt.subplot(6,2,6); plt.gray(); plt.imshow(grad_y)

plt.subplot(6,2,7); plt.gray(); plt.imshow(grad_abs)

threshold = grad_abs.min() + 0.5 * (grad_abs.max()-grad_abs.min())
plt.subplot(6,2,8); plt.gray(); plt.imshow(grad_abs>threshold)

plt.subplot(3,1,3); plt.gray(); plt.imshow(grad_abs)
# visualize gradients:
f = 16 ; scale = 20
for r,c in np.ndindex((img.shape[0]//f,img.shape[0]//f)):
    x0 = c*f; y0 = r*f
    i = grad_abs[y0:y0+f,x0:x0+f].argmax()
    dy, dx = np.unravel_index(i, (f,f))
    x = x0+dx; y = y0+dy
    if grad_abs[y,x] > threshold:
        plt.arrow(x, y, grad_x[y,x]*scale, grad_y[y,x]*scale,
                  color='yellow', head_width=10)
    
plt.show()

# Color

* Load the image `peppers.png`.

## Color

* No exercise sheet on color.
* Instead, recap of CV-06 here in the practice session

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

img = plt.imread('peppers.png')

plt.imshow(img)
plt.title(f"shape: {img.shape} ({img.dtype})")
plt.show()

### Task 1: 

* Display the red, green, and blue channel (in red, green, and blue respectively).
* Convert the image in the HSV colorspace. Display the H, S, and V channels (hint: there exists an `'hsv'` color map).

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

img = plt.imread('peppers.png')

plt.figure(figsize=(12,10))
plt.figure(figsize=(12,10))
plt.subplot(2,2,1); plt.axis('off'); plt.imshow(img)
# YOUR CODE HERE
plt.show()

In [ ]:
from matplotlib import colors

img_hsv = colors.rgb_to_hsv(img)

plt.figure(figsize=(12,10))
plt.subplot(2,2,1); plt.axis('off'); plt.imshow(img)
# YOUR CODE HERE
plt.show()

# Camera

OpenCV (module `cv2`) provides access to your webcam:

In [ ]:
%matplotlib inline
import imageio
import matplotlib.pyplot as plt

reader = imageio.get_reader('<video0>')
img = reader.get_next_data()
reader.close()

plt.imshow(img)
plt.title("Frame: {}, type={}".format(frame.shape, frame.dtype))
plt.show()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import cv2

camera = cv2.VideoCapture(0)
ok, frame = camera.read()
camera.release()

plt.figure(figsize=(12,6))
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
plt.title("Frame ({}): {}, type={}".format(ok, frame.shape, frame.dtype))
plt.show()

## Exercise:

1. Take two snapshots and create a difference image for motion detection (CV-03, slide 26)

In [ ]:
import time
input("Press enter for the first image ...")
camera = cv2.VideoCapture(0)
time.sleep(.5)
ok, frame1 = camera.read()
camera.release()

input("Press enter for the second image ...")
time.sleep(.5)
camera = cv2.VideoCapture(0)
ok, frame2 = camera.read()
camera.release()

diff = frame1
# YOUR CODE HERE

plt.figure(figsize=(12,6))
plt.gray()
plt.subplot(1,3,1); plt.imshow(f1); plt.title('Snapshot 1')
plt.subplot(1,3,2); plt.imshow(f2); plt.title('Snapshot 2')
plt.subplot(1,3,3); plt.imshow(diff); plt.title('Difference image')
plt.show()

## cvloop

* [`cvloop`](https://github.com/shoeffner/cvloop) is a tool developed by Sebastian Höffner, a former CV tutor.
* it allows to embed a camera stream into a notebook

In [ ]:
from cvloop import cvloop

cvloop()

In [ ]:
from cvloop import cvloop

def invert(frame):
    return 255-frame

cvloop(function=invert, side_by_side=True)

## Exercises
1. apply edge detection to the video stream
1. background subtraction: implement a simple form of background subtraction by first taking a reference picture without foreground object and then substract the background from the videostream.
1. color tracking: track a colored object moving in front of the camera and mark it in the video stream. You may also extend your solution and use the object to "paint" on the screen.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from skimage import color
from cvloop import cvloop

def my_edges(frame):
    edges = frame
    # YOUR CODE HERE
    return edges

# Run cvloop
cvloop(function=my_edges)

In [ ]:
%matplotlib notebook
import cv2
import numpy as np
from matplotlib import pyplot as plt
from skimage import color
from cvloop import cvloop

def my_background_subtraction(frame):
    global background_gray
    result = frame
    # YOUR CODE HERE
    return result

camera = cv2.VideoCapture(0)
assert camera, "No Capture device!"

input("Press enter to take the empty reference image ...")
ok, background = camera.read()
assert ok, "Image capture failed ({})!".format(ok)
background_gray = color.rgb2gray(background)
camera.release()

input("Now start the video stream ...")
cvloop(function=my_background_subtraction)

In [ ]:
%matplotlib notebook
import numpy as np
import skimage.color
from cvloop import cvloop

hue = .3 # green

def my_painter(frame):
    global hue
    frame_hsv = color.rgb2hsv(frame)
    # YOUR CODE HERE
    return frame

cvloop(function=my_painter)